# Libraries

In [ ]:
import numpy as np
import pandas as pd

# Visualisations
#import missingno
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Regression models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

# Model slection
from sklearn.model_selection import train_test_split


# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


# Other
from scipy import stats
import math
import pickle

# EDA

In [ ]:
train_df = pd.read_csv('df-train_set.csv')
test_df = pd.read_csv('df-test_set.csv')

test_ID = test_df['Index']
test_df = test_df.drop('Index', axis=1)

In [ ]:
train_df = train_df[(train_df['Commodities'] == 'APPLE GOLDEN DELICIOUS')]
train_df.drop(['Date'], axis= 1, inplace =True)
test_df.drop(['Date'], axis= 1, inplace =True)
#train_df['Date'] = pd.to_datetime(train_df['Date'],  format = '%B')
#train_df.drop(['Container'], axis= 1, inplace =True)

In [ ]:
#train_df = train_df[(train_df['Commodities'] == 'APPLE GOLDEN DELICIOUS')]
#train_df.drop(['Date'], axis= 1, inplace =True)
#test_df.drop(['Date'], axis= 1, inplace =True)
#train_df['Date'] = pd.to_datetime(train_df['Date'],  format = '%B')
#train_df.drop(['Container'], axis= 1, inplace =True)

In [ ]:
train_df.head()

In [ ]:
#Drop column low_price and High_price

In [ ]:
train_df.describe()

In [ ]:
train_df.dtypes

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.shape


# Dummy Variables


In [ ]:
df_dummies = pd.get_dummies(train_df)

df_dummies.columns = [col.replace(" ","_") for col in df_dummies.columns]
df_dummies.drop(['Commodities_APPLE_GOLDEN_DELICIOUS'], axis= 1, inplace =True)

df_dummies.head()

# Correlation and Variable selection

In [ ]:
#Avarage must be the last column as dependent
#reorder columns
column_titles = [col for col in df_dummies.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
df_dummies=df_dummies.reindex(columns=column_titles)

In [ ]:
# Target correlations with features
corrs = pd.DataFrame(df_dummies.corr()['avg_price_per_kg']).rename(columns = {'avg_price_per_kg':'Correlation'})
corrs.sort_values(by='Correlation',ascending=False, inplace=True)
corrs

In [ ]:
#spliting variables into x and y
X = df_dummies.drop('avg_price_per_kg', axis = 1)
Y = df_dummies['avg_price_per_kg']

# Training Data

In [ ]:
# create scaler object
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) #no response or Y needed


X_standardise = pd.DataFrame(X_scaled,columns=X.columns)
X_standardise.head()

In [ ]:
X_standardise.shape

In [ ]:
#y_test y_train

# Models

In [ ]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
#Train test and split
X_train, X_test, y_train, y_test = train_test_split(X_standardise, Y, test_size = 0.2, random_state=42)

#lasso model
lasso = Lasso(alpha=0.01)
#using linear regression model
model = LinearRegression()
#using ridge model
ridge = Ridge()

# Train the LASSO model
lasso.fit(X_train, y_train)

# Train the model
model.fit(X_train, y_train)
    
#train Ridge model
ridge.fit(X_train, y_train)

#print(train_pred)

In [ ]:
# Make predictions and evalute
X_train_p = model.predict(X_train)
X_test_p = model.predict(X_test)
train_lasso = lasso.predict(X_train)

train_ridge = ridge.predict(X_train)

In [ ]:
#using the test data
test_df = test_df[(test_df['Commodities'] == 'APPLE GOLDEN DELICIOUS')]



X_t = pd.get_dummies(test_df)
X_t.columns = [col.replace(" ","_") for col in X_t.columns]
X_t.drop(['Commodities_APPLE_GOLDEN_DELICIOUS'], axis= 1, inplace =True)

#X_t.head()

In [ ]:
#

# create scaler object

X_scaled = scaler.fit_transform(X_t) #no response or Y needed
X_real= pd.DataFrame(X_scaled,columns=X_t.columns)
X_real.head()

In [ ]:
X_real.shape

In [ ]:
#test_Tree = dec_tree.predict(X_real)

In [ ]:
test_lasso = lasso.predict(X_real)

test_pred =  model.predict(X_real)



In [ ]:
test_pred

In [ ]:
test_lasso

In [ ]:
test_Tree

In [ ]:
# create submission dataframe
# Create Dataframe of Order_No and Time from Pickup to Arrival
submission = pd.DataFrame(
    {'Index': test_ID,
     'avg_price_per_kg': test_Tree
    })

In [ ]:
submission.to_csv("subpredict_Tree.csv", index=False)

# Evaluating Model